### Extract Data from PDF Files with Python

In [ ]:
# !pip install PDFPlumber
import pdfplumber
with pdfplumber.open("38_Septiembre_2023.pdf") as temp:
   first_page = temp.pages[0]
   print(first_page.extract_table())

In [ ]:

with pdfplumber.open("38_Septiembre_2023.pdf") as temp:
   first_page = temp.pages[0]
   print(first_page.extract_text())

In [ ]:
text = first_page.extract_text()
lines = text.split('\n')
filtered_lines = [line for line in lines if 'Fecha' in line]
print('\n'.join(filtered_lines))

In [ ]:
print(filtered_lines)

In [ ]:
import os

root_dir = os.getcwd()

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            file_path = os.path.join(dirpath, filename)
            print(file_path)

In [ ]:
# !pip install PDFPlumber
import pdfplumber
with pdfplumber.open("38_Septiembre_2023.pdf") as temp:
   first_page = temp.pages[0]
   print(first_page.extract_table())

In [ ]:
# Declaración de un dataframe vacio para la carga
import pandas as pd
tabla = first_page.extract_table()
encabezados = tabla[0]
df = pd.DataFrame(columns=encabezados)

In [ ]:
# Se carga las filas del dataframe
for fila in tabla:
    df.loc[len(df)] = fila
print("Filas cargadas: {}".format(len(df))) 
df =df.drop(0)

In [ ]:
df

In [ ]:
b = first_page.extract_text()
nroFact  =b[0:15]
print("Nro Factura: {}".format(nroFact)) 
df['NroFactura']=nroFact

In [ ]:
text = first_page.extract_text()
lines = text.split('\n')
filtered_lines = [line for line in lines if 'Fecha y hora' in line]
# print('\n'.join(filtered_lines))

str(filtered_lines)
fechafact =str(filtered_lines)[27:37]
print("Fecha Emision: {}".format(fechafact))


In [ ]:
df['FechaEmision']=fechafact

In [ ]:
df

In [ ]:
df = df.drop(range(2, len(df), 1),axis=0)

In [ ]:
df

In [ ]:
df.dtypes